## Load Database

In [2]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("sqlite:///../plantation.db")

query = "SELECT * FROM plantation LIMIT 5"

pd.read_sql(query, engine)


,date,region,production_ton,operational_cost,sales_revenue,inventory_stock,profit
0,2024-01-01,Sulawesi,183,7977.0,17231,336,9254
1,2024-01-02,Sumatra,125,5602.0,7009,380,1407
2,2024-01-03,Sulawesi,103,6134.0,7260,458,1126
3,2024-01-04,Sulawesi,193,5522.0,13038,300,7516
4,2024-01-05,Sumatra,111,6268.0,16865,345,10597


## Bussines Question

*Q1 — Profit rata-rata per region*

In [3]:
query = """
SELECT region, AVG(profit) AS avg_profit
FROM plantation
GROUP BY region
ORDER BY avg_profit DESC;
"""

pd.read_sql(query, engine)

,region,avg_profit
0,Sulawesi,8437.748031
1,Sumatra,8211.611111
2,Kalimantan,8210.383929


*Q2 — Total profit per bulan*

In [4]:
query = """
SELECT strftime('%Y-%m', date) AS month,
SUM(profit) AS total_profit
FROM plantation
GROUP BY month
ORDER BY month;
"""

pd.read_sql(query, engine)


,month,total_profit
0,2024-01,253254
1,2024-02,227301
2,2024-03,271630
3,2024-04,233314
4,2024-05,277462
5,2024-06,233971
6,2024-07,255113
7,2024-08,230385
8,2024-09,280384
9,2024-10,279295


*Region ranking berdasarkan profit*

In [5]:
query = """
SELECT region,
SUM(profit) AS total_profit,
RANK() OVER (ORDER BY SUM(profit) DESC) AS rank
FROM plantation
GROUP BY region;
"""

pd.read_sql(query, engine) 

,region,total_profit,rank
0,Sulawesi,1071594,1
1,Sumatra,1034663,2
2,Kalimantan,919563,3


## Insight

*Rata-rata profit yang dipunya oleh setiap region di range 8200-8400 dengan region sulawasi yang memilki profit tertinggi*

*Total profit tertinggi terjadi pada bulan Desember dengan total profit sebesar 90000an*


## Case Level 2 Business Scenario

*Case 1 — Region dengan biaya tertinggi*

In [11]:
query = """
    SELECT region,
        AVG(operational_cost) AS avg_cost
    FROM plantation
    GROUP BY region
    ORDER BY avg_cost DESC;
"""

pd.read_sql(query, engine)

,region,avg_cost
0,Sulawesi,5221.160550
1,Sumatra,5176.558649
2,Kalimantan,5080.047468


*Case 2--Profit margin per region*

In [13]:
#Margin =  profit/ revenue
query = """
    SELECT region,
       SUM(profit) * 1.0 / SUM(sales_revenue) AS profit_margin
    FROM plantation
    GROUP BY region
    ORDER BY profit_margin DESC;
"""
pd.read_sql(query, engine)

,region,profit_margin
0,Kalimantan,0.619670
1,Sulawesi,0.618588
2,Sumatra,0.613855


*Case 3 — Bulan terburuk dalam setahun*

In [21]:
query = """
SELECT strftime('%Y-%m', date) AS month,
    SUM(profit) AS total_profit
FROM plantation
GROUP BY month
ORDER BY total_profit ASC
LIMIT 1;
"""
pd.read_sql(query, engine)

,month,total_profit
0,2024-02,227301


*Case 4 — Hari dengan biaya ekstrem*

In [22]:
query = """
SELECT date, region, operational_cost
FROM plantation
ORDER BY operational_cost DESC
LIMIT 5;
"""
pd.read_sql(query, engine)

,date,region,operational_cost
0,2024-05-08,Sulawesi,7997.0
1,2024-05-01,Sulawesi,7989.0
2,2024-08-30,Sulawesi,7986.0
3,2024-01-01,Sulawesi,7977.0
4,2024-07-06,Sulawesi,7971.0


*Case 5 — Ranking produksi vs profit*

*Apakah produksi tinggi selalu profit tinggi?*

In [25]:
query = """
    SELECT region,
        SUM(production_ton) AS total_production,
        SUM(profit) AS total_profit,
        RANK() OVER (ORDER BY SUM(profit) DESC) AS profit_rank
    FROM plantation
    GROUP BY region;
"""
pd.read_sql(query, engine)

,region,total_production,total_profit,profit_rank
0,Sulawesi,17995,1071594,1
1,Sumatra,17356,1034663,2
2,Kalimantan,15730,919563,3


## Insight

*Region Sulawesi memiliki biaya operasi tertinggi sekaligus menjadi region yang paling menghasilkan profit* 

*BUlan Februari menunjukan profit terendah, diperlukan analisis lebih lanjut jika dimunkinkan bisa juga melakukan audit terkait dengan biaya operasi yang digunakan pada bulan tersebut*

*Region Kalimantan memiliki total production dan total profit yang paling kecil dibandingkan dengan region yang lain , perlu dilakukan analisa yang mendalam untuk mengetahui apa yang dapat dilkaukan untuk menaikan produksi di region kalimantan* 